In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


In [29]:
train_file = 'blp25_hatespeech_subtask_1A_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1A_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1A_dev_test.tsv'

In [30]:
#tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# Load train/val/test DataFrames
train_df = pd.read_csv(train_file, sep="\t")
dev_df = pd.read_csv(validation_file , sep="\t")
test_df = pd.read_csv(test_file, sep="\t")





In [4]:
l2id = {
    'None': 0,
    'Religious Hate': 1,
    'Sexism': 2,
    'Political Hate': 3,
    'Profane': 4,
    'Abusive': 5
}
id2l = {v: k for k, v in l2id.items()}


def clean_label(x):
    # handle missing or NaN → "None"
    if pd.isna(x) or x == 'None':
        return 'None'
    # already list-like e.g. ['Abusive']
    if isinstance(x, list):
        return x[0] if len(x) > 0 else 'None'
    # string cases like "[]" or "[Abusive]" or "[Political Hate]"
    x = x.strip("[]").strip()
    if x == "":
        return 'None'
    return x


def process_df(df):
    # Ensure labels are proper lists
    df["label"] = df["label"].apply(clean_label)
    df["label"] = df["label"].fillna("None")
    # Now create binary label
    df["toxic"] = df["label"].apply(lambda x: 0 if x == "None" else 1)
    df["label_id"] = df["label"].map(l2id)

    

  

    return df

train_df = process_df(train_df)
dev_df  = process_df(dev_df)

train_df

,id,text,label,toxic,label_id
0,147963,ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...,None,0,0
1,214275,ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...,None,0,0
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Abusive,1,5
3,821985,চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...,None,0,0
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Abusive,1,5
...,...,...,...,...,...
35517,790325,তইওয়ানের এত ক্ষমতা হয়নি যে এক টুকরো জায়গা নষ্ট...,None,0,0
35518,328377,চুরের ঘরের চুর হালা,Profane,1,4
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,Abusive,1,5
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,Abusive,1,5


In [5]:

# Display the result
print(train_df[['text', 'toxic']].head())

                                                text  toxic
0  ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...      0
1  ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...      0
2          অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে      1
3  চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...      0
4  এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...      1


In [6]:
train_df.columns

Index(['id', 'text', 'label', 'toxic', 'label_id'], dtype='object')

In [7]:
# Filter for toxic comments only
toxic_train_df = train_df[train_df['toxic'] == 1].copy()
toxic_dev_df = dev_df[dev_df['toxic'] == 1].copy()


# Display the result
print(toxic_train_df[['text', 'label']].head())

                                                 text    label
2           অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে  Abusive
4   এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...  Abusive
7   ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...  Abusive
11  মুসলিম বাচ্চাগুলো বাচ্চা পেরে পেরে গোটা পৃথিবী...  Profane
15                                   বাড়ি বাল ফালায়ছে  Profane


In [25]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding  # <-- Import the data collator
)
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# --- 1. Load Tokenizer and Model ---
model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


# def truncate_long_texts(df, max_chars=10000):
#     df['text'] = df['text'].astype(str).apply(lambda x: x[:max_chars])
#     return df

# train_df = truncate_long_texts(train_df)
# dev_df = truncate_long_texts(dev_df)

# --- 2. Create Datasets ---
train_dataset_1 = Dataset.from_pandas(train_df[['text', 'toxic']].rename(columns={'toxic': 'label'}))
dev_dataset_1 = Dataset.from_pandas(dev_df[['text', 'toxic']].rename(columns={'toxic': 'label'}))

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=512)



train_dataset_1 = train_dataset_1.map(tokenize, batched=True)
dev_dataset_1 = dev_dataset_1.map(tokenize, batched=True)

# Set the format to torch tensors and specify columns
train_dataset_1.set_format("torch", columns=["input_ids", "attention_mask", "label"])
dev_dataset_1.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# --- 3. Define Training ---

# Create a data collator that will dynamically pad the batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics_binary(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds), "f1": f1_score(p.label_ids, preds)}

training_args_1 = TrainingArguments(
    output_dir="./results_model_1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir="./logs_1",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer_1 = Trainer(
    model=model_1,
    args=training_args_1,
    train_dataset=train_dataset_1,
    eval_dataset=dev_dataset_1,
    compute_metrics=compute_metrics_binary,
    data_collator=data_collator, # <-- Add the data collator here
)

# --- 4. Start Fine-tuning ---
trainer_1.train()
trainer_1.save_model("./final_model_1")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

/users/t/p/tprama/.local/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.461400,0.467177,0.784236,0.744098
2,0.369500,0.497184,0.797771,0.763501
3,0.263400,0.669088,0.787022,0.749883


In [9]:
toxic_train_df

,id,text,label,toxic,label_id
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Abusive,1,5
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Abusive,1,5
7,786609,ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...,Abusive,1,5
11,124917,মুসলিম বাচ্চাগুলো বাচ্চা পেরে পেরে গোটা পৃথিবী...,Profane,1,4
15,432369,বাড়ি বাল ফালায়ছে,Profane,1,4
...,...,...,...,...,...
35514,668158,কৃষি মন্ত্রীর সারের ব্যবসা করে তাই এ অবস্থা,Political Hate,1,3
35518,328377,চুরের ঘরের চুর হালা,Profane,1,4
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,Abusive,1,5
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,Abusive,1,5


In [10]:
toxic_dev_df

,id,text,label,toxic,label_id
0,166449,ইন্ডিয়া কি মাছ ধরা বন্ধ রাখছেএক নদীতে দুইনীতি ...,Political Hate,1,3
1,267692,লক্ষ টাকা ঘুষ দিয়ে অযোগ্য আর দায়িত্বহীন মানস...,Abusive,1,5
3,939131,আর কতো শিখবে আমার সোনার ছেলেরা এগুলো কে টাকা দ...,Abusive,1,5
4,210284,কি সাংঘাতিক ভাই রে তুই,Abusive,1,5
5,712332,লঞ্চ মালিকদের অভিশপ্ত চক্ষু পদ্মা সেতুর উপর,Abusive,1,5
...,...,...,...,...,...
2496,653048,কিরে মানিক চোরা তুইও আছোস,Abusive,1,5
2503,121961,দেশের সবই তো চুরি হয়ে যাচ্ছে আর চোরদের ধরার কো...,Abusive,1,5
2504,555021,ফকিনি বাংলা দেশ কারেন্ট নাই,Abusive,1,5
2505,858412,কাকু সামনে চশমা থাকবে কিন্তু গ্লাস থাকবেনাপাছা...,Profane,1,4


In [11]:

l2id = {
   
    'Religious Hate': 0,
    'Sexism': 1,
    'Political Hate': 2,
    'Profane': 3,
    'Abusive': 4
}
id2l = {v: k for k, v in l2id.items()}


toxic_train_df["label"] = toxic_train_df["label"].map(l2id)
toxic_train_df['label'].unique()

toxic_dev_df["label"] = toxic_dev_df["label"].map(l2id)
toxic_dev_df['label'].unique()

array([2, 4, 3, 1, 0])

In [12]:
toxic_train_df

,id,text,label,toxic,label_id
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,4,1,5
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,4,1,5
7,786609,ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...,4,1,5
11,124917,মুসলিম বাচ্চাগুলো বাচ্চা পেরে পেরে গোটা পৃথিবী...,3,1,4
15,432369,বাড়ি বাল ফালায়ছে,3,1,4
...,...,...,...,...,...
35514,668158,কৃষি মন্ত্রীর সারের ব্যবসা করে তাই এ অবস্থা,2,1,3
35518,328377,চুরের ঘরের চুর হালা,3,1,4
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,4,1,5
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,4,1,5


In [13]:

# import torch
# device = torch.device("cpu")
# model_2.to(device)


In [24]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding  # <-- Import the data collator
)
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# --- 1. Load Tokenizer and Model ---
model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_2 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


# def truncate_long_texts(df, max_chars=10000):
#     df['text'] = df['text'].astype(str).apply(lambda x: x[:max_chars])
#     return df

# train_df = truncate_long_texts(train_df)
# dev_df = truncate_long_texts(dev_df)

# --- 2. Create Datasets ---
train_dataset_2 = Dataset.from_pandas(toxic_train_df[['text', 'label']])
dev_dataset_2 = Dataset.from_pandas(toxic_dev_df[['text', 'label']])

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=512)



train_dataset_2 = train_dataset_2.map(tokenize, batched=True)
dev_dataset_2 = dev_dataset_2.map(tokenize, batched=True)

# Set the format to torch tensors and specify columns
train_dataset_2.set_format("torch", columns=["input_ids", "attention_mask", "label"])
dev_dataset_2.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# --- 3. Define Training ---

# Create a data collator that will dynamically pad the batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics_multiclass(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1_weighted": f1_score(p.label_ids, preds, average="weighted"),
    }

training_args_2 = TrainingArguments(
    output_dir="./results_model_2",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir="./logs_1",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer_2 = Trainer(
    model=model_2,
    args=training_args_2,
    train_dataset=train_dataset_2,
    eval_dataset=dev_dataset_2,
    compute_metrics=compute_metrics_multiclass,
    data_collator=data_collator, # <-- Add the data collator here
)

# --- 4. Start Fine-tuning ---
trainer_2.train()
trainer_2.save_model("./final_model_2")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/15568 [00:00<?, ? examples/s]

Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

/users/t/p/tprama/.local/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted
1,0.852200,0.635245,0.760603,0.758042
2,0.603000,0.616558,0.762488,0.757744
3,0.444100,0.716113,0.762488,0.757096
4,0.288100,0.934421,0.744581,0.740676
5,0.208100,1.131335,0.749293,0.748000
6,0.135200,1.324743,0.726673,0.725817
7,0.112400,1.614933,0.727615,0.727433
8,0.069900,1.779849,0.741753,0.740247
9,0.054000,1.958204,0.734213,0.730717
10,0.041000,2.078136,0.742696,0.739061


In [31]:
from transformers import pipeline
from sklearn.metrics import classification_report
import numpy as np

# --- 1. Load the fine-tuned models with truncation ---
pipe_1 = pipeline(
    "text-classification",
    model="./final_model_1",
    tokenizer=tokenizer,
    device=-1,   # force CPU
    truncation=True,
    max_length=512
)

pipe_2 = pipeline(
    "text-classification",
    model="./final_model_2",
    tokenizer=tokenizer,
    device=-1,
    truncation=True,
    max_length=512,
    return_all_scores=True
)

# --- 2. Run the pipeline on the test set ---
predictions = []
for text in test_df['text']:
    # Model 1: Is it toxic?
    result_1 = pipe_1(text, truncation=True, max_length=512)[0]
    is_toxic = 1 if result_1['label'] == 'LABEL_1' else 0
    
    pred_labels = [0] * pipe_2.model.config.num_labels  # default: all zero
    
    if is_toxic:
        # Model 2: Which toxic class? (highest score = predicted label)
        result_2 = pipe_2(text, truncation=True, max_length=512)[0]
        best_label = max(result_2, key=lambda x: x['score'])
        pred_labels = [1 if r['label'] == best_label['label'] else 0 for r in result_2]
    
    predictions.append(pred_labels)

y_pred = np.array(predictions)
print(y_pred.shape)


Device set to use cpu
Device set to use cpu
/users/t/p/tprama/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


(2512, 5)


In [32]:
test_df


,id,text
0,879187,শুভ কামনা রইল বাংলাদেশ জন্য ইনশাআল্লাহ জয় হবে
1,316919,গোয়া মারা দিয়ে আছে বাংলাদেশ মাদারচোদ নিউজ করে ...
2,916242,ভাইয়া আপনি অভিনেতা হইয়েন না না হলে সবাই বাচ্...
3,786824,আমাদেরো তাই দেখছি
4,47284,পুলিশ কতটা টাকা নিয়ে
...,...,...
2507,776466,সত্য কথা তেতু লাগে
2508,849227,এই ফকিননি মাগীটা আর কত নাটক দেখাবে
2509,532697,দেখো আজকে কার ফিটনেস কোথায় দাঁড়িয়েছে তুমি চ...
2510,861411,ছোট ভাইটির পাসে থাকুন গেম ভিড়িও বানাই


In [33]:
import numpy as np
import pandas as pd

# Your mapping
id2l = {
    0: 'Religious Hate',
    1: 'Sexism',
    2: 'Political Hate',
    3: 'Profane',
    4: 'Abusive'
}


l2id = {
    'None': 0,
    'Religious Hate': 1,
    'Sexism': 2,
    'Political Hate': 3,
    'Profane': 4,
    'Abusive': 5
}

# Example y_pred
# y_pred = np.array([[0,0,0,0,0],[0,0,0,1,0],[1,0,0,0,0]])

def decode_labels(row):
    indices = np.where(row == 1)[0]
    if len(indices) == 0:
        return "None"
    # If multiple labels, join them with comma
    return ", ".join([id2l[i] for i in indices])

# Convert predictions into a DataFrame column
df = pd.DataFrame()
df["Predicted_Label"] = [decode_labels(row) for row in y_pred]

print(df.head())


  Predicted_Label
0            None
1         Profane
2            None
3            None
4            None


In [34]:
df

,Predicted_Label
0,None
1,Profane
2,None
3,None
4,None
...,...
2507,None
2508,Profane
2509,Profane
2510,None


In [35]:
test_df['label']=df['Predicted_Label']
test_df['model']='bangla-bert'
test_df

,id,text,label,model
0,879187,শুভ কামনা রইল বাংলাদেশ জন্য ইনশাআল্লাহ জয় হবে,None,bangla-bert
1,316919,গোয়া মারা দিয়ে আছে বাংলাদেশ মাদারচোদ নিউজ করে ...,Profane,bangla-bert
2,916242,ভাইয়া আপনি অভিনেতা হইয়েন না না হলে সবাই বাচ্...,None,bangla-bert
3,786824,আমাদেরো তাই দেখছি,None,bangla-bert
4,47284,পুলিশ কতটা টাকা নিয়ে,None,bangla-bert
...,...,...,...,...
2507,776466,সত্য কথা তেতু লাগে,None,bangla-bert
2508,849227,এই ফকিননি মাগীটা আর কত নাটক দেখাবে,Profane,bangla-bert
2509,532697,দেখো আজকে কার ফিটনেস কোথায় দাঁড়িয়েছে তুমি চ...,Profane,bangla-bert
2510,861411,ছোট ভাইটির পাসে থাকুন গেম ভিড়িও বানাই,None,bangla-bert


In [36]:
#test_df['model']='bert-base-multilingual-cased'

In [37]:
test_df.columns

Index(['id', 'text', 'label', 'model'], dtype='object')

In [38]:
test_df = test_df[['id', 'label', 'model']]
test_df

,id,label,model
0,879187,None,bangla-bert
1,316919,Profane,bangla-bert
2,916242,None,bangla-bert
3,786824,None,bangla-bert
4,47284,None,bangla-bert
...,...,...,...
2507,776466,None,bangla-bert
2508,849227,Profane,bangla-bert
2509,532697,Profane,bangla-bert
2510,861411,None,bangla-bert


In [39]:
test_df.to_csv("final_bert_v2.tsv", sep="\t", index=False)
print("Saved to final_ensemble.tsv")

Saved to final_ensemble.tsv
